# Modeling - Random Forest

In [15]:
from sklearn import (
    feature_extraction,
    ensemble,
    metrics,
    model_selection,
    preprocessing,

)
import pandas as pd
from nltk.tokenize import word_tokenize

In [16]:
# Importing the dataset

bncc_cleaned_df = pd.read_csv('/home/pdsmoreau/data-science/tera/bncc-classifier/data/curateddf_bncc_model_two_curated_pedro_04032022.csv')

bncc_cleaned_df.head()

,Unnamed: 0,id,questions_clean,target_enc
0,0,3486670,base conhecimentos sobre compositor john cage ...,0
1,1,3343927,achar número somado 2 igual inverso” equações ...,10
2,4,3691951,chuva ácida fenômeno provocado poluição atmosf...,11
3,6,1744035,pagina nr 54 livro didático,7
4,7,2525602,chiaroscuro,0


In [17]:
bncc_cleaned_df = bncc_cleaned_df.astype({"questions_clean": str})

In [18]:
# removing registers with zero chars
bncc_cleaned_df["words_count"] = bncc_cleaned_df["questions_clean"].apply(len)
bncc_cleaned_filtered_df = bncc_cleaned_df[bncc_cleaned_df["words_count"] != 0]
bncc_cleaned_filtered_df

,Unnamed: 0,id,questions_clean,target_enc,words_count
0,0,3486670,base conhecimentos sobre compositor john cage ...,0,346
1,1,3343927,achar número somado 2 igual inverso” equações ...,10,65
2,4,3691951,chuva ácida fenômeno provocado poluição atmosf...,11,97
3,6,1744035,pagina nr 54 livro didático,7,27
4,7,2525602,chiaroscuro,0,11
...,...,...,...,...,...
76442,99994,2447693,leia reportagem responda questãosobre pergunta...,9,99
76443,99995,4330761,responder questão leia trecho contoprefácio hi...,9,1458
76444,99996,3249373,pais surgiu skate,3,17
76445,99998,4150214,machos fêmeas mosquitos gênero aedes vivem loc...,2,415


### Splitting the dataset into traind and test and applying Bag of Words

In [19]:
# sklearn patterns names
X = bncc_cleaned_filtered_df.drop(
    ["id", "words_count", "Unnamed: 0", "target_enc"], axis=1
)
y = bncc_cleaned_filtered_df["target_enc"]
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, random_state=1
)

In [20]:
# bow vect
vectBOW = feature_extraction.text.CountVectorizer()
X_train_trans = vectBOW.fit_transform(X_train["questions_clean"])
X_test_trans = vectBOW.transform(X_test["questions_clean"])
print(X_train_trans.shape, X_test_trans.shape)

(57335, 140264) (19112, 140264)


### Grid Search

In [21]:
# Armazenando os parâmetros a serem otimizados pelo Grid Search em um dicionário

param_grid = {'max_depth': [3, 4, 5, 6, 7]}

In [22]:
est = ensemble.RandomForestClassifier(n_estimators=1000, random_state=42)

grid_search = model_selection.GridSearchCV(est, param_grid, n_jobs=-1, scoring='accuracy') # O Grid Search teve como parâmetro 
# a maximização da acurácia
rf_grid_search = grid_search.fit(X_train_trans, y_train)
y_pred_class = rf_grid_search.predict(X_test_trans)

In [23]:
print(metrics.classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       991
           1       0.00      0.00      0.00       760
           2       0.00      0.00      0.00      1816
           3       0.00      0.00      0.00       586
           4       0.00      0.00      0.00       104
           5       1.00      0.04      0.08      1285
           6       0.92      0.03      0.05      2385
           7       0.86      0.08      0.14      2263
           8       1.00      0.34      0.51      1990
           9       0.52      0.70      0.60      2896
          10       0.22      0.99      0.36      3194
          11       0.00      0.00      0.00       842

    accuracy                           0.32     19112
   macro avg       0.38      0.18      0.14     19112
weighted avg       0.50      0.32      0.23     19112



/home/pdsmoreau/.local/share/virtualenvs/bncc-classifier-Qtat0hV3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pdsmoreau/.local/share/virtualenvs/bncc-classifier-Qtat0hV3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pdsmoreau/.local/share/virtualenvs/bncc-classifier-Qtat0hV3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div